# 任务2

In [2]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('用户新增预测挑战赛公开数据/train.csv')
test_data = pd.read_csv('用户新增预测挑战赛公开数据/test.csv')

train_data['common_ts'] = pd.to_datetime(train_data['common_ts'], unit='ms')
test_data['common_ts'] = pd.to_datetime(test_data['common_ts'], unit='ms')

In [3]:
def udmap_onethot(d):
    v = np.zeros(9)
    if d == 'unknown':
        return v
    
    d = eval(d)
    for i in range(1, 10):
        if 'key' + str(i) in d:
            v[i-1] = d['key' + str(i)]
            
    return v

train_udmap_df = pd.DataFrame(np.vstack(train_data['udmap'].apply(udmap_onethot)))
test_udmap_df = pd.DataFrame(np.vstack(test_data['udmap'].apply(udmap_onethot)))

train_udmap_df.columns = ['key' + str(i) for i in range(1, 10)]
test_udmap_df.columns = ['key' + str(i) for i in range(1, 10)]

In [9]:
# print(train_data)
# print('aa')
# print(train_udmap_df)

          uuid  eid                          udmap               common_ts  \
0            0   26  {"key3":"67804","key2":"650"} 2023-07-18 09:44:28.244   
1            1   26  {"key3":"67804","key2":"484"} 2023-07-11 13:42:21.469   
2            2    8                        unknown 2023-07-15 07:49:53.040   
3            3   11                        unknown 2023-07-16 00:36:55.688   
4            4   26  {"key3":"67804","key2":"650"} 2023-07-16 07:15:51.442   
...        ...  ...                            ...                     ...   
620351  620351    2      {"key4":"14","key5":"14"} 2023-07-10 12:07:30.527   
620352  620352    8                        unknown 2023-07-16 06:51:42.305   
620353  620353   41    {"key1":"274","key2":"446"} 2023-07-09 08:41:18.584   
620354  620354   26  {"key3":"67804","key2":"650"} 2023-07-18 04:02:40.171   
620355  620355   41   {"key1":"1398","key2":"289"} 2023-07-13 08:03:18.023   

        x1  x2  x3   x4   x5  x6  ...  key4  key5  key6  key7  

In [4]:
train_data = pd.concat([train_data, train_udmap_df], axis=1)
test_data = pd.concat([test_data, test_udmap_df], axis=1)

In [5]:
train_data['eid_freq'] = train_data['eid'].map(train_data['eid'].value_counts())
test_data['eid_freq'] = test_data['eid'].map(train_data['eid'].value_counts())

train_data['eid_mean'] = train_data['eid'].map(train_data.groupby('eid')['target'].mean())
test_data['eid_mean'] = test_data['eid'].map(train_data.groupby('eid')['target'].mean())

In [6]:
train_data['udmap_isunknown'] = (train_data['udmap'] == 'unknown').astype(int)
test_data['udmap_isunknown'] = (test_data['udmap'] == 'unknown').astype(int)

In [7]:
train_data['common_ts_hour'] = train_data['common_ts'].dt.hour
test_data['common_ts_hour'] = test_data['common_ts'].dt.hour

In [8]:
print(train_data)
print(test_data)

          uuid  eid                          udmap               common_ts  \
0            0   26  {"key3":"67804","key2":"650"} 2023-07-18 09:44:28.244   
1            1   26  {"key3":"67804","key2":"484"} 2023-07-11 13:42:21.469   
2            2    8                        unknown 2023-07-15 07:49:53.040   
3            3   11                        unknown 2023-07-16 00:36:55.688   
4            4   26  {"key3":"67804","key2":"650"} 2023-07-16 07:15:51.442   
...        ...  ...                            ...                     ...   
620351  620351    2      {"key4":"14","key5":"14"} 2023-07-10 12:07:30.527   
620352  620352    8                        unknown 2023-07-16 06:51:42.305   
620353  620353   41    {"key1":"274","key2":"446"} 2023-07-09 08:41:18.584   
620354  620354   26  {"key3":"67804","key2":"650"} 2023-07-18 04:02:40.171   
620355  620355   41   {"key1":"1398","key2":"289"} 2023-07-13 08:03:18.023   

        x1  x2  x3   x4   x5  x6  ...  key4  key5  key6  key7  

In [7]:
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(
    train_data.drop(['udmap', 'common_ts', 'uuid', 'target'], axis=1),
    train_data['target']
)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [8]:
pd.DataFrame({
    'uuid': test_data['uuid'],
    'target': clf.predict(test_data.drop(['udmap', 'common_ts', 'uuid'], axis=1))
}).to_csv('submit.csv', index=None)